#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 1:  Retrieval Models: Boolean + Vector Space

### 100 points [5% of your final grade]

### Due: Thursday, February 2 by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine. In particular, there are three main learning objectives: (i) the basics of tokenization (e.g. stemming, case-folding, etc.) and its effect on information retrieval; (ii) basics of index building and Boolean retrieval; and (iii) basics of the Vector Space model and ranked retrieval.

*Submission Instructions:* To submit your homework, rename this notebook as lastname_firstinitial_hw#.ipynb. For example, my homework submission would be: caverlee_j_hw1.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 5 at 11:59pm.

## Dataset

We provide the dataset, [southpark_scripts.zip](https://www.dropbox.com/s/6rzfsbn97s8vwof/southpark_scripts.zip), which includes scripts for episodes of the first twenty seasons of the TV show South Park. You will build an inverted index over these scripts where each episode should be treated as a single document. There are 277 episodes (documents) to index and search on.

## (20 points) Part 1: Parsing

First, you should tokenize documents using **whitespaces and punctuations as delimiters** but do not remove stop words. Your parser needs to also provide the following two confgiuration options:
* Case-folding
* Stemming: use [nltk Porter stemmer](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter)

Please note that you should stick to the stemming package listed above. Otherwise, given the same query, the results generated by your code can be different from others.

In [1]:
from nltk.stem.porter import PorterStemmer
from collections import defaultdict
import re
import glob
import math
import operator
# configuration options
use_stemming = True # or false
use_casefolding = True # or false
stemmer = PorterStemmer()

filesPath = "*.txt"

num_files = 0
for fileName in glob.iglob(filesPath):
    num_files += 1
#print num_files

cache = {}
def tokenize(line, words):
    temp_list = re.split('\W+', line)
    for word in temp_list:
        if word == "":
            continue
        if use_casefolding == True:
            word = word.lower()
        if use_stemming == True:
            if word not in cache:
                cache[word] = stemmer.stem(word)
            word = cache[word]
        words.append(word)

In [2]:
# Your parser function here. It will take the two option variables above as the parameters
# add cells as needed to organize your code

words = set()
for fileName in glob.iglob(filesPath):
    outFile = open(fileName, 'r')
    temp_wordList = []
    for line in outFile:
        tokenize(line, temp_wordList)
    for word in temp_wordList:
        words.add(word)
        
print len(words)

17143


### Observations

Once you have your parser working, you should report here the size of your dictionary under the four cases. That is, how many unique tokens do you have with stemming on and casefolding on? And so on. You should fill in the following

* Stemming + Casefolding       = 17143
* Stemming + No Casefolding    = 17368
* No Stemming + Casefolding    = 23806
* No Stemming + No Casefolding = 29550


## (40 points) Part 2: Boolean Retrieval

In this part you build an inverted index to support Boolean retrieval. We only require your index to  support AND queries. In other words, your index does not have to support OR, NOT, or parentheses. Also, we do not explicitly expect to see AND in queries, e.g., when we query **great again**, your search engine should treat it as **great** AND **again**.

Example queries:
* Rednecks
* Troll Trace
* Respect my authority
* Respect my authoritah
* Respected my authority

In [3]:
# build the index here
# add cells as needed to organize your code
dict = defaultdict(list)
for fileName in glob.iglob(filesPath):
    outFile = open(fileName, 'r')
    words = []
    for line in outFile:
        tokenize(line, words)
    for word in words:
        dict[word].append(fileName)
#print "hello"

In [4]:
search_text = raw_input('Boolean Search:')
# search for the input using your index and print out ids of matching documents
words = []
tokenize(search_text, words)
print words
results = set()
for fileName in glob.iglob(filesPath):
    results.add(fileName)
for word in words:
    if word in dict:
        results = results.intersection(dict[word])
    else:
        results.clear()
        break

for fileName in results:
    #print fileName
    print (re.findall('([0-9]+)', fileName))[0]

Boolean Search:yellow
['yellow']
0303
1409
1011
1604
0205
0715
1207
0408
2005
1206
1105
1312
1401
1613
0108


### Observations

Q. When we use stemming and casefolding, is the result different from the result when we do not use them? 
Ans - Yes, the results are different. Casefolding improves matching a lot in the sense that if a word is present in different orientation in the doc and query, it will still get cached. Similarly, stemming is the cort part of matching here. Many different forms of a word exist. Stemming ensures that those forms of word match, by converting a word into its root form.

Q. Do you find cases where you prefer stemming? Or not? Or cases where you prefer casefolding? Or Not? Write down your observations below.
Ans - Stemming is preferred when queries have words which are different derivation of same word like
Respect my authority = [u'respect', 'my', u'author']
Respected my authority = [u'respect', 'my', u'author']

CaseFolding is very useful when your query and your doc may be in different cases. A lot of potential matches can be missed if no casefolding is performed.
OF - ['OF'] - no casefolding
OF - ['of'] - with casefolding.
Stemming will be bad in case of abbreviations.

Ans1 - Yes, the results are different. Casefolding improves matching a lot in the sense that if a word is present in different orientation in the doc and query, it will still get cached. Similarly, stemming is the cort part of matching here. Many different forms of a word exist. Stemming ensures that those forms of word match, by converting a word into its root form.

Ans2 - Stemming is preferred when queries have words which are different derivation of same word like Respect my authority = [u'respect', 'my', u'author'] . Respected my authority = [u'respect', 'my', u'author'].    
CaseFolding is very useful when your query and your doc may be in different cases. A lot of potential matches can be missed if no casefolding is performed. OF - ['OF'] - no casefolding  while OF - ['of'] - with casefolding. 
Stemming will be bad in case of abbreviations.

### BONUS: Phrase Queries

Your search engine needs to also (optionally) support phrase queries of arbitrary length. Use quotes in a query to tell your search engine this is a phrase query. Again, we don't explicitly type AND in queries and never use OR, NOT, or parentheses.

In [7]:
# Your code here
def phrase_query(search_text):
    results = []
    for fileName in glob.iglob(filesPath):
        with open(fileName, 'r') as content_file:
            content = content_file.read()
            content = content.lower()
            search_text = search_text.lower()
            if search_text in content:
                results.append(fileName)
    return results

In [10]:
search_text = raw_input('Phrase Query:')
if re.match(r'".*"', search_text):
    search_text = search_text[1:-1]
    results = phrase_query(search_text)
    for fileName in results:
        #print fileName
        print (re.findall('([0-9]+)', fileName))[0]

Phrase Query:Stan's pinewood derby


## (40 points) Part 3: Ranked Retrieval

In this part, your job is to support queries over an index that you build. This time you will use the vector space model plus cosine similarity to rank documents.

**TFIDF:** For the document vectors, use the standard TFIDF scores. That is, use the log-weighted term frequency $1+log(tf)$; and the log-weighted inverse document frequency $log(\frac{N}{df})$. For the query vector, use simple weights (the raw term frequency). For example:
* query: troll $\rightarrow$ (1)
* query: troll trace $\rightarrow$ (1, 1)

**Output:**
For a given query, you should rank all the 277 documents but you only need to output the top-5 documents (i.e. document ids) plus the cosine score of each of these documents. For example:

* result1 - score1
* result2 - score2
* result3 - score3
* result4 - score4
* result5 - score5

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!

In [11]:
# build the vector space index here
# add cells as needed to organize your code

def magnitude(u):
    return math.sqrt(sum(u[i]*u[i] for i in range(len(u))))

def magnitude_map(u):
    return math.sqrt(sum(u[i]*u[i] for i in u))

def dot(u, v):
    return sum(u[i]*v[i] for i in range(len(u)))

def normalize(v):
    vmag = magnitude(v)
    if vmag == 0:
        return v
    return [ float(v[i])/vmag  for i in range(len(v)) ]

def normalize_map(v):
    vmag = magnitude_map(v)
    if vmag == 0:
        return v
    for i in v:
        v[i] = float(v[i])/vmag 
    return v

In [12]:
search_text = raw_input('Ranked Search:')
# search for the input and print the top 5 document ids along with their associated cosine scores.
query_words = []
words_set = set()
doc_vec_dict = defaultdict(list)
query_vec = []
results = defaultdict(int)

# query tokenization
tokenize(search_text, query_words)
words_set = set(query_words)
visited = []
for word in query_words:
    if word in visited:
        continue
    query_vec.append(query_words.count(word))
    visited.append(word)
query_vec = normalize(query_vec)


for fileName in glob.iglob(filesPath):
    doc_vec = {}
    for word in dict:
        #Term freqency
        num_relevant_docs = len(set(dict[word]))
        idf = math.log10((float(num_files)/num_relevant_docs))
        
        tf = 0
        if dict[word].count(fileName) > 0:
            tf = 1 + math.log10(dict[word].count(fileName))
        
        
        doc_vec[word] = tf*idf
    
    doc_vec = normalize_map(doc_vec)
    relevant_doc_vec = []
    visited = []
    for word in query_words:
        if word in visited:
            continue
        if word not in dict:
            relevant_doc_vec.append(0)
        else:
            relevant_doc_vec.append(doc_vec[word])
        visited.append(word)
        
    results[fileName] = dot(relevant_doc_vec, query_vec)
    
sorted_x = sorted(results.items(), key=operator.itemgetter(1), reverse=True)
count = 0
for item in sorted_x:
    if count == 5:
        break
    print str((re.findall('([0-9]+)', item[0]))) + "-" + str(item[1])
    count = count + 1

Ranked Search:foo
['0409']-0.111545238548
['1609']-0.0847375597958
['0509']-0.0606180344219
['1902']-0.0
['1608']-0.0


## How we grade

The grader will randomly pick 5-10 queries to test your program. You are welcome to discuss the results returned by your search engine with others on Piazza.